

**Problem Statement:** Create a convolutional neural network that trains to 99.9% accuracy on these images, which cancels training upon hitting this training accuracy threshold. We will be using the happy or sad dataset, which contains 80 images of emoji-like faces, 40 happy and 40 sad.

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

**Load the Data**

In [ ]:
from tensorflow.keras.preprocessing.image import load_img

base_dir = "./data/"
happy_dir = os.path.join(base_dir, "happy/")
sad_dir = os.path.join(base_dir, "sad/")

print("Sample happy image:")
plt.imshow(load_img(f"{os.path.join(happy_dir, os.listdir(happy_dir)[0])}"))
plt.show()

print("\nSample sad image:")
plt.imshow(load_img(f"{os.path.join(sad_dir, os.listdir(sad_dir)[0])}"))
plt.show()

**Get Input Details**

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array

# Load the first example of a happy face
sample_image  = load_img(f"{os.path.join(happy_dir, os.listdir(happy_dir)[0])}")

# Convert the image into its numpy array representation
sample_array = img_to_array(sample_image)

print(f"Each image has shape: {sample_array.shape}")

print(f"The maximum pixel value used is: {np.max(sample_array)}")

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.999:
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True

**Image Data Preprocessing using the ImageDataGenerator API**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def image_generator():
    train_datagen = ImageDataGenerator(rescale = 1/255)

    train_generator = train_datagen.flow_from_directory(directory='./data/',
                                                        target_size=(150, 150),
                                                        batch_size=10,
                                                        class_mode='binary')

    return train_generator

# Save the generator in a variable
gen = image_generator()

**Design the Model**

In [ ]:
from tensorflow.keras import optimizers, losses

def train_happy_sad_model(train_generator):

    # Instantiate the callback
    callbacks = myCallback()

    # Define the model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units = 128, activation = 'relu'),
        tf.keras.layers.Dense(units = 1, activation = "sigmoid")


    ])

    # Compile the model
    # Select a loss function compatible with the last layer of your network
    model.compile(loss=losses.binary_crossentropy,
                  optimizer="adam",
                  metrics=['accuracy'])


    # Train the model
    history = model.fit(x=train_generator,
                        epochs=15,
                        callbacks=myCallback()
                       )

    return history

In [ ]:
hist = train_happy_sad_model(gen)
print(f"Your model reached the desired accuracy after {len(hist.epoch)} epochs")

**This model reached the desired accuracy after 10 epochs**